In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Show all output for a cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [3]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [4]:
from src.inference import get_feature_store
from datetime import datetime, timedelta
import pandas as pd

# Get the current datetime in UTC
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# Define the 28-day batch window
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=40)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")

# Load the feature view
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME,
    version=config.FEATURE_VIEW_VERSION
)

# Fetch the full time range (+1 day buffer on both sides)
ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)

# Filter strictly within the range, localize to naive timestamps
ts_data = ts_data[ts_data["hour"].between(fetch_data_from, fetch_data_to)]
ts_data = ts_data.sort_values(["start_station_id", "hour"]).reset_index(drop=True)
ts_data["hour"] = ts_data["hour"].dt.tz_localize(None)

# Transform to sliding window features
from src.data_utils import transform_ts_data_into_features
features = transform_ts_data_into_features(
    ts_data, window_size=24*28, step_size=23
)

2025-05-11 10:17:15,889 INFO: Initializing external client
2025-05-11 10:17:15,890 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 10:17:16,755 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Fetching data from 2025-04-01 14:17:15.886568+00:00 to 2025-05-11 13:17:15.886568+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.86s) 


In [5]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-11 10:17:23,208 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 10:17:23,214 INFO: Initializing external client
2025-05-11 10:17:23,214 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 10:17:23,802 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009


Downloading: 0.000%|          | 0/324120 elapsed<00:00 remaining<?

In [6]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [7]:
predictions["hour"] = current_date.ceil('h')
predictions

,start_station_id,predicted_demand,hour
0,5329.03,21.0,2025-05-11 15:00:00+00:00
1,5329.03,12.0,2025-05-11 15:00:00+00:00
2,5905.14,11.0,2025-05-11 15:00:00+00:00
3,5905.14,5.0,2025-05-11 15:00:00+00:00
4,6140.05,23.0,2025-05-11 15:00:00+00:00
5,6140.05,10.0,2025-05-11 15:00:00+00:00


In [8]:
from src.inference import get_feature_store

feature_group = get_feature_store().get_or_create_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1,
    description="Predictions from LGBM Model",
    primary_key=["start_station_id", "hour"],
    event_time="hour",
)

2025-05-11 10:17:25,950 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 10:17:25,956 INFO: Initializing external client
2025-05-11 10:17:25,957 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 10:17:26,516 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009


In [9]:
feature_group.insert(predictions, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 6/6 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: citi_bike_hourly_model_prediction_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1231009/jobs/named/citi_bike_hourly_model_prediction_1_offline_fg_materialization/executions


(Job('citi_bike_hourly_model_prediction_1_offline_fg_materialization', 'SPARK'),
 None)